Nombre 1: Daniela Nicoleta Boldureanu

Nombre 2: Adrián Nieto Moya

Nº de grupo: 10

![](https://informatica.ucm.es/themes/ucm16/media/img/logo.png)

# Bases de datos NoSQL


#### Práctica 5

El siguiente código asegura que la biblioteca Redis está en el sistema. Si no está la instalará, pero OJO: puede que haya que reiniciar el notebook (Kernel + Restart)

In [1]:
import sys
# redis
try:
    import redis
    print("Redis está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user redis

Redis está en el sistema!


Modificar la variable `redisconfPath` con el path al fichero redisconf.py, que hemos debido modificar con nuestras claves. Si todo va bien mostrará True. Si no, avisar al profesor

In [2]:
redisconfPath = "C:/Users/Daniela/Documents/dudu/nosql/redis/redisconf.py"

#redisconfPath = "c:/hlocal/"

import sys
sys.path.insert(0, redisconfPath)
import redisconf


import  redis



pool = redis.ConnectionPool(host=redisconf.CONEXION, password=redisconf.PASSWD, 
                port=redisconf.PUERTO, decode_responses=True, db=0)
r = redis.Redis(connection_pool=pool)
r.ping()


True

# Preparación

Para hacer la práctica debemos tener ejecutándose en otra pestaña el notebook `sensor.ipynb`. Este notebook simula un sistema de varios sensores (no nos importa cuántos) en el que que,  aproximadamente cada segundo (esto tampoco tenemos que tenerlo en cuenta), se toma un valor de uno de los sensores y se emite mediante 3 claves Redis:

* sensor: el número de sensor seleccionado
* time: un valor que se va incrementando y representa el tiempo
* value: el valor emitido por el sensor

En ocasiones, para algunos valores de  `sensor`y un `time` puede que la clave `value` no exista; representa un error de lectura.

Nótese que lo ideal sería que cada segundo se tomaran los valores de todos los sensores, pero aquí, por simplicidad, cada segundo se toman los datos de uno al azar. Lo único que podemos asegurar es que el `time` siempre se irá incrementando (no sabemos en cuanto) y nunca se repite, que `sensor`tendrá el número de sensor, y que `value`, si existe, es la medida tomada de ese sensor en ese tiempo.

### Ejercicio 1 [5 puntos]

Queremos hacer la media de 5 valores consecutivos emitidos por el notebook sensor. Cosas a tener en cuenta:

* Serán 5 valores consecutivos desde el primero disponible cuando ejecutemos este fragmento de código; no importa los que se hayan emitido antes

* Ir mostrando por pantalla los valores leídos de redis para facilitar la depuración (y la corrección)

* Podemos asumir que este fragmento de código es más rápido que el del sensor, que emite cada aprox. un segundo. Es decir, no tenemos que preocuparnos ni comprobar por si hemos perdido ningún valor. Sin embargo, sí debemos comprobar por ejemplo si `value` existe o no (si no existe toma el valor None). Si no existe lógicamente no se contará como uno de los 5 valores consecutivos, se ignorará.

* Somos conscientes de que al hacer la media de 5 valores consecutivos estaremos mezclando valores de sensores distintos; es una primera aproximación que mejoraremos en el siguiente ejercicio. En este nos da igual de qué sensor proviene el valor, solo nos importan los valores 'time' y 'valor'.

* El valor tendrá que ser convertido a int, en el modo de uso de redis que tenemos aunque sea un int se recoge como un string.

In [20]:

r.flushall()
cuantos = 5
i=0
suma=0
anterior=-1
while i<cuantos:
    v =  r.mget(['time', 'value'])
    sensorTime = v[0]
    sensorValue = v[1]
    
    if sensorValue!=None and sensorTime!=anterior:
        suma=suma+int(sensorValue)
        anterior=sensorTime
        i=i+1
        print(sensorValue)
   
        
print(suma/cuantos)
        



8000
90
40
70
400
1720.0


### Ejercicio 2 [5 puntos]


El código anterior mezcla valores de diferentes sensores; ahora queremos obtener los 5 primeros valores consecutivos que encontremos para cada sensor (no hace falta la media, solo queremos recopilarlos). 

La idea es análoga al código anterior pero usando listas y conjuntos para recopilar la información. En particular:

* llevaremos un conjunto de redis con nombre `sensores` con los nombres de sensores que ya han aparecido
* Además, habrá una lista por cada sensor con sus `cuantos` valores. La lista tendrá como clave el nombre del sensor.

Cada vez que aparezca un dato de un sensor (que no sea un valor None):

* Añadimos el nombre al conjunto `sensores`
* Comprobamos la longitud de la lista cuyo nombre es el nombre del sensor (método `llen`). Si es menor que `cuantos`, añadimos el nuevo valor. Si ya tiene `cuantos`, lo ignoramos.
* Finalmente, comprobamos si a) `sensores` tiene un cardinal de al menos `num_sensores`, y b) si la longitud de las listas de todos los sensores acumulados en `sensores` es `cuantos`. Si se cumplen ambas condiciones hemos acabado; si no habrá que seguir iterando.

* Al final, mostraremos los valores recopilados de cada lista cuyo nombre aparece en `sensores`

Notas: 

* En cada lista se van introduciendo valores por la izquierda
* No hace falta crear el conjunto vacío, ni la lista vacía; ambas se crean en redis la primera vez que se utilizan.
* En este caso el número de sensor no conviene convertirlo a entero, así se puede usar directamente como clave para la lista, que tiene que ser un string

In [12]:
cuantos = 5 # valores a guardar
num_sensores  = 3 # número de sensores
r.flushall()
anterior=-1
seguir=True

while seguir or r.scard("sensores")<num_sensores:
    v =  r.mget(['time', 'value', 'sensor'])
    sensorTime = v[0]
    sensorValue = v[1]
    sensor=v[2]
    if sensorValue!=None and sensorTime!=anterior:
        r.sadd("sensores",sensor)
        anterior=sensorTime
        if r.llen(sensor)<cuantos:
            r.lpush(sensor, sensorValue)
    seguir=False
    for i in r.smembers("sensores"):
        if r.llen(i)<cuantos or seguir:
            seguir=True
        else:
            seguir=False


for i in r.smembers("sensores"):
    print("Sensor"+i+":")
    print(r.lrange(i,0,-1))


Sensor3:
['6000', '8000', '4000', '7000', '1000']
Sensor2:
['800', '100', '400', '700', '900']
Sensor1:
['80', '10', '80', '20', '10']


No olvidar subir el notebook al campus con los nombres de los componentes del grupo (arriba)